In [27]:
import os
import torch # For building the networks 
import torchtuples as tt # Some useful functions
from torch import nn
import torch.nn.functional as F

from pycox.datasets import support
import numpy as np
import cox
from cox import CoxPH
from pycox.preprocessing.feature_transforms import OrderedCategoricalLong
import pandas as pd
import model
from model import Cox_cluster
import pandas as pd
import numpy as np
import sklearn_pandas
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
import seaborn as sns

from pycox.evaluation import EvalSurv
sns.set_style("white")


from pycox.preprocessing.feature_transforms import OrderedCategoricalLong

import cox
from cox import CoxPH
from pycox.preprocessing.feature_transforms import OrderedCategoricalLong

import model
from model import Cox_cluster

np.random.seed(43)
_ = torch.manual_seed(9)
file_name_train = [file for file in os.listdir(r"path/var2_5_sample/train") if file != '.DS_Store']
file_name_test = [file for file in os.listdir(r"path/var2_5_sample/test") if file != '.DS_Store']
file_name_val = [file for file in os.listdir(r"path/var2_5_sample/val") if file != '.DS_Store']

file_name_re_train = [file for file in os.listdir(r"path/var2_5_sample/re_train") if file != '.DS_Store']
file_name_re_test = [file for file in os.listdir(r"path/var2_5_sample/re_test") if file != '.DS_Store']
file_name_re_val = [file for file in os.listdir(r"path/var2_5_sample/re_val") if file != '.DS_Store']


def function_work(x):
    y = x.rsplit('.', 2)[-2]
    return ('log' not in x, int(y) if y.isdigit() else float('inf'), x)

csvFiles = file_name_train
csvFiles_train=sorted(sorted(csvFiles, key=function_work, reverse=False),key=len, reverse=False)
csvFiles = file_name_test
csvFiles_test=sorted(sorted(csvFiles, key=function_work, reverse=False),key=len, reverse=False)
csvFiles = file_name_val
csvFiles_val=sorted(sorted(csvFiles, key=function_work, reverse=False),key=len, reverse=False)
csvFiles = file_name_re_train
csvFiles_re_train=sorted(sorted(csvFiles, key=function_work, reverse=False),key=len, reverse=False)
csvFiles = file_name_re_test
csvFiles_re_test=sorted(sorted(csvFiles, key=function_work, reverse=False),key=len, reverse=False)
csvFiles = file_name_re_val
csvFiles_re_val=sorted(sorted(csvFiles, key=function_work, reverse=False),key=len, reverse=False)
arr=[]
arr = [0 for i in range(len(csvFiles_test))] 
#csvFiles_test
arr_ibs=[]
arr_ibs = [0 for i in range(len(csvFiles_test))] 


In [29]:
import time

# Initialize variables to track total time
total_time = 0
for i in range(len(csvFiles_train)):
    start_time = time.time()  # Start tracking time
    file_path_train=r"path/var2_5_sample/train"+"/"+csvFiles_train[i]
    data1_train=pd.read_csv(file_path_train)
    file_path_test=r"path/var2_5_sample/test"+"/"+csvFiles_test[i]
    data1_test=pd.read_csv(file_path_test)
    file_path_val=r"path/var2_5_sample/val"+"/"+csvFiles_val[i]
    data1_val=pd.read_csv(file_path_val)
    file_path_re_train=r"path/var2_5_sample/re_train"+"/"+csvFiles_re_train[i]
    re_train=pd.read_csv(file_path_re_train)
    file_path_re_test=r"path/var2_5_sample/re_test"+"/"+csvFiles_re_test[i]
    re_test=pd.read_csv(file_path_re_test)
    file_path_re_val=r"path/var2_5_sample/re_val"+"/"+csvFiles_re_val[i]    
    re_val=pd.read_csv(file_path_re_val)
    df_test = data1_test
    df_train = data1_train
    df_val = data1_val
    cols_std = ['x1', 'x2', 'x3', 'x4','x5']

    standardize = [([col], StandardScaler()) for col in cols_std]


    x_mapper = DataFrameMapper(standardize)

    x_train = x_mapper.fit_transform(df_train).astype('float32')
    x_test = x_mapper.transform(df_test).astype('float32')

    re_test = re_test
    re_train = re_train

    x_train = x_mapper.fit_transform(df_train).astype('float32')

    x_test = x_mapper.transform(df_test).astype('float32')
    x_val = x_mapper.transform(df_val).astype('float32')

    re_test = re_test
    re_train = re_train
    re_val=re_val

    cols_bin_re=list(re_train.columns)[1:]
    leave_re = [(col, None) for col in cols_bin_re]



    x_mapper_re= DataFrameMapper(leave_re)
    re_train = x_mapper_re.fit_transform(re_train).astype('float32')
 
    re_test = x_mapper_re.transform(re_test).astype('float32')
    re_val = x_mapper_re.transform(re_val).astype('float32')

    get_target = lambda df: (df['o'].values, df['delta'].values)
    y_train = get_target(df_train)
    y_val = get_target(df_val)
    durations_test, events_test = get_target(df_test)
    val = x_val,re_val, y_val
    optimizer = tt.optim.AdamWR(decoupled_weight_decay=0.001, cycle_eta_multiplier=0.5,
                            cycle_multiplier=2)
    In_Nodes= x_train.shape[1]
    num_nodes = [64, 64]
    Pathway_Nodes=num_nodes[0]
    Hidden_Nodes=num_nodes[1]
    Out_Nodes= 1
    batch_norm = True
    dropout = 0.2
    output_bias = False
    batch_size =128


    net= Cox_cluster(In_Nodes, Pathway_Nodes, Hidden_Nodes, Out_Nodes,)
    model = cox.CoxPH(net, optimizer)

    epochs = 100
    callbacks = [tt.callbacks.EarlyStopping()]
    verbose = True


    batch_size =128
    lrfind = model.lr_finder(x_train, re_train,y_train,batch_size, tolerance=100)

    print(lrfind.get_best_lr())


    model.optimizer.set_lr(lrfind.get_best_lr())
    log = model.fit(x_train, re_train,y_train, batch_size, epochs, callbacks, verbose, val_data=val,
                val_batch_size=batch_size)

    baseline = model.compute_baseline_hazards(batch_size=100000)
    surv = model.predict_surv_df(x_test,re_test,baseline_hazards_=baseline)
    surv = model.predict_surv_df(x_test,re_test,baseline_hazards_=baseline)
    ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
    arr[i]=ev.concordance_td()
    print(ev.concordance_td())
    time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
    arr_ibs[i]=ev.integrated_brier_score(time_grid)
    print(arr_ibs[i])
    end_time = time.time()
    elapsed_time = end_time - start_time  # Calculate time taken for this round
    total_time += elapsed_time  # Accumulate the total time
    print(f"Round {i+1} took {elapsed_time:.2f} seconds.")

None
None
0.00023101297000831686
None
None
0:	[0s / 0s],		train_loss: 4.4308,	val_loss: 4.4146
1:	[0s / 0s],		train_loss: 4.4269,	val_loss: 4.4145
2:	[0s / 0s],		train_loss: 4.4316,	val_loss: 4.4145


/Users/ruiwenzhou/Downloads/deep_learning_clustered-main 0223/deep_learning_cluster/tt_Model.py:703: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(t

3:	[0s / 0s],		train_loss: 4.4284,	val_loss: 4.4145
4:	[0s / 0s],		train_loss: 4.4266,	val_loss: 4.4145
5:	[0s / 0s],		train_loss: 4.4329,	val_loss: 4.4145
6:	[0s / 0s],		train_loss: 4.4277,	val_loss: 4.4145
7:	[0s / 0s],		train_loss: 4.4310,	val_loss: 4.4145
8:	[0s / 0s],		train_loss: 4.4335,	val_loss: 4.4145
9:	[0s / 0s],		train_loss: 4.4318,	val_loss: 4.4145
10:	[0s / 0s],		train_loss: 4.4274,	val_loss: 4.4145
11:	[0s / 0s],		train_loss: 4.4259,	val_loss: 4.4145
12:	[0s / 0s],		train_loss: 4.4291,	val_loss: 4.4145
13:	[0s / 0s],		train_loss: 4.4274,	val_loss: 4.4145
14:	[0s / 0s],		train_loss: 4.4342,	val_loss: 4.4145
15:	[0s / 0s],		train_loss: 4.4316,	val_loss: 4.4145
16:	[0s / 0s],		train_loss: 4.4301,	val_loss: 4.4145
17:	[0s / 0s],		train_loss: 4.4317,	val_loss: 4.4145
18:	[0s / 0s],		train_loss: 4.4256,	val_loss: 4.4145
19:	[0s / 0s],		train_loss: 4.4304,	val_loss: 4.4145
20:	[0s / 0s],		train_loss: 4.4322,	val_loss: 4.4145
21:	[0s / 0s],		train_loss: 4.4297,	val_loss: 4.4145


/Users/ruiwenzhou/Downloads/deep_learning_clustered-main 0223/deep_learning_cluster/tt_Model.py:703: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(t

1:	[0s / 0s],		train_loss: 4.4122,	val_loss: 4.3985
2:	[0s / 0s],		train_loss: 4.4123,	val_loss: 4.3984
3:	[0s / 0s],		train_loss: 4.4133,	val_loss: 4.3983
4:	[0s / 0s],		train_loss: 4.4070,	val_loss: 4.3982
5:	[0s / 0s],		train_loss: 4.4106,	val_loss: 4.3981
6:	[0s / 0s],		train_loss: 4.4131,	val_loss: 4.3981
7:	[0s / 0s],		train_loss: 4.4115,	val_loss: 4.3981
8:	[0s / 0s],		train_loss: 4.4124,	val_loss: 4.3980
9:	[0s / 0s],		train_loss: 4.4104,	val_loss: 4.3979
10:	[0s / 0s],		train_loss: 4.4117,	val_loss: 4.3978
11:	[0s / 0s],		train_loss: 4.4114,	val_loss: 4.3978
12:	[0s / 0s],		train_loss: 4.4158,	val_loss: 4.3978
13:	[0s / 0s],		train_loss: 4.4138,	val_loss: 4.3978
14:	[0s / 0s],		train_loss: 4.4156,	val_loss: 4.3978
15:	[0s / 0s],		train_loss: 4.4127,	val_loss: 4.3977
16:	[0s / 0s],		train_loss: 4.4126,	val_loss: 4.3977
17:	[0s / 0s],		train_loss: 4.4131,	val_loss: 4.3976
18:	[0s / 0s],		train_loss: 4.4107,	val_loss: 4.3975
19:	[0s / 0s],		train_loss: 4.4106,	val_loss: 4.3975
20

/Users/ruiwenzhou/Downloads/deep_learning_clustered-main 0223/deep_learning_cluster/tt_Model.py:703: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(t

1:	[0s / 0s],		train_loss: 4.4112,	val_loss: 4.3990
2:	[0s / 0s],		train_loss: 4.4157,	val_loss: 4.3989
3:	[0s / 0s],		train_loss: 4.4094,	val_loss: 4.3988
4:	[0s / 0s],		train_loss: 4.4129,	val_loss: 4.3987
5:	[0s / 0s],		train_loss: 4.4079,	val_loss: 4.3987
6:	[0s / 0s],		train_loss: 4.4128,	val_loss: 4.3987
7:	[0s / 0s],		train_loss: 4.4135,	val_loss: 4.3986
8:	[0s / 0s],		train_loss: 4.4180,	val_loss: 4.3985
9:	[0s / 0s],		train_loss: 4.4116,	val_loss: 4.3985
10:	[0s / 0s],		train_loss: 4.4124,	val_loss: 4.3984
11:	[0s / 0s],		train_loss: 4.4116,	val_loss: 4.3984
12:	[0s / 0s],		train_loss: 4.4096,	val_loss: 4.3983
13:	[0s / 0s],		train_loss: 4.4139,	val_loss: 4.3983
14:	[0s / 0s],		train_loss: 4.4152,	val_loss: 4.3983
15:	[0s / 0s],		train_loss: 4.4118,	val_loss: 4.3983
16:	[0s / 0s],		train_loss: 4.4195,	val_loss: 4.3982
17:	[0s / 0s],		train_loss: 4.4124,	val_loss: 4.3982
18:	[0s / 0s],		train_loss: 4.4119,	val_loss: 4.3981
19:	[0s / 0s],		train_loss: 4.4140,	val_loss: 4.3981
20

In [31]:
import statistics
def Average(lst):
    return sum(lst) / len(lst)
  

average_c_index = Average(arr)
print(average_c_index)
average_ibs = Average(arr_ibs)
print(average_ibs)
sd_c_index = statistics.stdev(arr)
print(sd_c_index)
sd_ibs = statistics.stdev(arr_ibs)
print(sd_ibs)

0.7184329063995776
0.19008737471727247
0.010411834578432758
0.0072282339555758465


In [33]:
average_time_per_round = total_time / len(csvFiles_test)
print(f"Average computation time per round: {average_time_per_round:.2f} seconds")

Average computation time per round: 3.51 seconds
